In [3]:
import datetime
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import phik
import scipy.stats as stats
import seaborn as sns
import statsmodels.api as sm
import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'
sns.set_theme(context='talk', style='whitegrid', palette='deep')
plt.rcParams['figure.figsize'] = 10, 7
plt.rcParams['font.size'] = 20
plt.rcParams['axes.labelsize'] = 25
plt.rcParams['figure.titlesize'] = 32
plt.rcParams['axes.titlesize'] = 32
plt.rcParams['savefig.format'] = 'pdf'
plt.rcParams['figure.autolayout'] = 'true'
plt.rcParams['figure.frameon'] = 'false'
plt.rcParams['axes.spines.left'] = 'false'
plt.rcParams['axes.spines.right'] = 'false'
plt.rcParams['axes.spines.top'] = 'false'
plt.rcParams['legend.fancybox'] = 'false'
plt.rcParams['axes.spines.bottom'] = 'false'
plt.rcParams['font.size'] = 20
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
# для графиков, где надо много цветов, юзайте воть:
sns.set_palette(sns.color_palette('deep'))
# а по дефолту воть:
sns.set_palette(sns.color_palette('BuGn_r', n_colors=10)[2::3])
pd.set_option('display.max_columns', 60)

In [4]:
feeding_details_22 = pd.read_csv('datasets/2022-feeding-tasks-details.csv', on_bad_lines='skip')
feeding_22 = pd.read_csv('datasets/2022-feeding-tasks.csv', on_bad_lines='skip')

feeding_details_23 = pd.read_csv('datasets/2023-feeding-tasks-details.csv', on_bad_lines='skip')
feeding_23 = pd.read_csv('datasets/2023-feeding-tasks.csv', on_bad_lines='skip')

feeding_details_24 = pd.read_csv('datasets/2024-feeding-tasks-details.csv', on_bad_lines='skip')
feeding_24 = pd.read_csv('datasets/2024-feeding-tasks.csv', on_bad_lines='skip')

feeding_details_25 = pd.read_csv('datasets/2025-feeding-tasks-details.csv', on_bad_lines='skip')
feeding_25 = pd.read_csv('datasets/2025-feeding-tasks.csv', on_bad_lines='skip')

In [5]:
monthly_feeding = pd.read_excel('datasets/Ekoniva_dataset.xlsx', sheet_name='Feeding')
herd_metrics = pd.read_excel('datasets/Ekoniva_dataset.xlsx', sheet_name='Herd maintenance').replace('-', np.nan)
production_indicators = pd.read_excel('datasets/Ekoniva_dataset.xlsx', sheet_name='Dairy indicators').replace('-', np.nan)

In [10]:
def merge_tables() -> pd.DataFrame:
    """
    Merges all feeding dataframes with all details dataframes
    returns: merged dataframe
    rtype: pd.DataFrame
    """
    feeding_22["year"] = "2022"
    feeding_23["year"] = "2023"
    feeding_24["year"] = "2024"
    feeding_25["year"] = "2025"
    feeding_details_22["year"] = "2022"
    feeding_details_23["year"] = "2023"
    feeding_details_24["year"] = "2024"
    feeding_details_25["year"] = "2025"

    feeding_all = pd.concat([feeding_22, feeding_23, feeding_24, feeding_25], ignore_index=True)
    feeding_details_all = pd.concat([feeding_details_22, feeding_details_23, feeding_details_24, feeding_details_25], ignore_index=True)
    
    return feeding_all.merge(feeding_details_all, on=["FeedingTaskID", "SectionID", "year", "PhysiologicalGroupID", "PhysiologicalGroupName"], how="left"), feeding_all, feeding_details_all

feeding_and_details, feeding_all, feeding_details_all = merge_tables()


In [17]:
feeding_details_all[feeding_details_all['IngredientName'].isin(["01.01", "02.01", "03.05", "12.01"])]


,FeedingTaskID,SectionID,PhysiologicalGroupID,PhysiologicalGroupName,IngredientID,IngredientName,IngredientType,PhysicalWeight_kg,year
3127222,Farms/EkoNiva1C.0db18646-1ce2-11ea-bbbf-b88303...,3,4,Д1,68,12.01,Forage,1816.000,2023
3127223,Farms/EkoNiva1C.0db18646-1ce2-11ea-bbbf-b88303...,3,4,Д1,70,03.05,Forage,1844.000,2023
3127236,Farms/EkoNiva1C.0db18646-1ce2-11ea-bbbf-b88303...,4,4,Д1,68,12.01,Forage,1924.000,2023
3127237,Farms/EkoNiva1C.0db18646-1ce2-11ea-bbbf-b88303...,4,4,Д1,70,03.05,Forage,1913.000,2023
3127250,Farms/EkoNiva1C.0db18646-1ce2-11ea-bbbf-b88303...,1,4,Д1,68,12.01,Forage,1205.000,2023
...,...,...,...,...,...,...,...,...,...
4083946,Farms/EkoNiva1C.0db18646-1ce2-11ea-bbbf-b88303...,1,4,Д1,70,03.05,Forage,613.772,2023
4083960,Farms/EkoNiva1C.0db18646-1ce2-11ea-bbbf-b88303...,2,4,Д1,134,02.01,Forage,298.588,2023
4083961,Farms/EkoNiva1C.0db18646-1ce2-11ea-bbbf-b88303...,2,4,Д1,70,03.05,Forage,613.316,2023
4083975,Farms/EkoNiva1C.0db18646-1ce2-11ea-bbbf-b88303...,7,4,Д1,134,02.01,Forage,290.602,2023


In [ ]:
import re
from collections import Counter
def group(feeding_and_details):
    def normalize_name(s):
        if pd.isna(s):
            return ""
        s = str(s).strip()
        s = s.replace("//", "/")
        s = re.sub(r"\s+", " ", s)
        s = s.lower()
        return s
    feeding_and_details['norm_ingr_name'] = feeding_and_details['IngredientName'].apply(normalize_name)
    code_re = re.compile(r'^\d+(?:\.\d+)+$') #Честно сам писал (100%)
    feeding_and_details['is_code'] = feeding_and_details['norm_ingr_name'].str.match(code_re)
    smth = pd.ExcelFile("datasets/Ekoniva_dataset.xlsx")
    nsi = smth.sheet_names[0]
    cultures = pd.read_excel(
        smth,
        sheet_name=nsi,
        usecols="B:C",
        skiprows=81,
        nrows=40 
    )
    cultures.columns = ["code", "name"]
    razdels = pd.read_excel(
        smth,
        sheet_name=nsi,
        usecols="E:H",
        skiprows=81,
        nrows=88  
    )
    razdels.columns = ["code", "region", "prop_farm_name", "farm_name"]
    feed_type = pd.read_excel(
        smth,
        sheet_name=nsi,
        usecols="B:C",
        skiprows=123,
        nrows=10
    )
    feed_type.columns = ["code", "feed_name"]
    cultures_map = cultures.set_index('code')['name'].to_dict()
    feed_type_map = feed_type.set_index('code')['feed_name'].to_dict()
    def decode_ingr_code(code):
        if pd.isna(code) or not isinstance(code, str):
            return code
        parts = code.split('.')
        if len(parts) < 4:
            return code
        try:
            culture_code = float(parts[2])
            feed_code = float(parts[3])
        except ValueError:
            return code    
        culture_name = cultures_map.get(culture_code, str(culture_code))
        feed_name = feed_type_map.get(feed_code, str(feed_code))
        return f"{culture_name} {feed_name}".lower()
    feeding_and_details['decoded_name'] = feeding_and_details.apply(
        lambda row: decode_ingr_code(row['norm_ingr_name']) if row['is_code'] else row['norm_ingr_name'],
        axis=1
    )
    GROUP_KEYWORDS = {
    # Основной корм/кормовая база (фураж) — для обеспечения клетчатки, объёма рациона
    "forage_bulk": [
        "силос", "сенаж", "сено", "люцерн", "солома", "зеленая масса", "зел масса",
        "сило", "сорго", "солома пшеницы", "солома ячменя", "сенокос", "сенок",
        "тюк", "тюкован", "травосмесь", "сенокуб", "сеносилос", "рожь", "трава",
        "солома", "солома", "soloma", "cилос"
    ],

    # Энергетические компоненты — зерно, крахмал, меласса, глицерин и пр.
    "energy_source": [
        "кукуруза", "пшениц", "ячмень", "овёс", "зерно", "corn", "wheat", "barley",
        "oats", "grain", "мука кукурузная", "мука ячменная", "кукуруза молотая",
        "дерть", "плющенка", "крахмал", "солод", "патока", "патока свекловичная",
        "меласса", "глицерин", "глицерин (glycerin)", "концентрат углевод", "корнаж", "карнаж",
        "глюгтен", "глютен", "корн", "зерносмесь", "солод", "тритикале"
    ],

    # Быстрые метаболические источники/антикетозные средства (используются при риске кетоза)
    "metabolic_support": [
        "глицерин", "глицерин (glycerin)", "пропиленгликоль", "пропиленгликоль (жидкий)",
        "propylene glycol", "глицерин", "глицерин", "энергетик"
    ],

    # Протеиновые компоненты
    "protein_source": [
        "шрот", "жмых", "соев", "соевый", "соя", "люпин", "горох", "бобы", "белок",
        "подсолн", "подсолнечный шрот", "экспандат", "экспеллер", "шрот соевый", "шр"
    ],

    # Жиры/защищённые жиры
    "fat_oils": [
        "жир", "масло", "жир защищённый", "масло соевое", "жир растительный", "жир животный",
        "пальмовое масло", "РЖК", "protected fat"
    ],

    # Минералы и витамины (профилактика дефицитов, костно-минеральный обмен)
    "minerals_vitamins": [
        "мел", "соль", "витамин", "минерал", "кальций", "фосфат", "извест", "Ca", "P",
        "NaCl", "CaCO3", "минсмесь", "витамины", "минералы", "ди кальций фосфат",
        "морская соль", "кальцит"
    ],

    # Буферы / рН регуляторы — профилактика ацидоза ()
    "buffers_ph": [
        "сода", "сода корм", "ниспользуются в рационеатрий бикарбонат", "буфер", "bicarbonate"
    ],

    # Электролиты / регидратация —  используются для профилактики и лечения дегидратации
    "electrolytes": [
        "электролит", "электролиты", "электролитный раствор", "electrolyte"
    ],

    # Кормовые добавки: пробиотики, ферменты, сорбенты, консерванты — поддержка рубца, профилактика
    "feed_additives_probiotics": [
        "дрожж", "пробиот", "пробиотик", "фермент", "энзим", "энзимы", "сорбент", "адсорбент",
        "консервант", "антиоксидант", "детокс", "детоксикант", "биотек", "аквасейф", "мегабуст",
        "холин", "аминокислота", "ферменты", "enzyme", "acid"
    ],

    # Комплексные премиксы/смеси/комбикорма — для молодых животных, концентраты
    "compound_feed_calf_replacements": [
        "комбикорм", "кк", "стартер", "престартер", "бустер милк", "кальвобустер", "зцм",
        "заменитель цельного молока", "ЗЦМ", "кормосмесь", "концентрат", "ПК", "complete feed"
    ],

    # Побочные продукты / отбросы пищевой промышленности
    "byproducts": [
        "жом", "барда", "жом свекловичный", "сыворотка", "дробина", "пивная дробина",
        "DDGS", "ддгс", "зерноотход", "остатк", "отруби", "побочка", "брикет жома"
    ],

    # Влажные сырьё / источники влаги (влияют на консистенцию и санитарное состояние)
    "wet_ingredients_water": [
        "вода", "влажн", "сыворотка молочная", "молоко", "H2O", "жидкость", "влажное сырье",
        "сыворотк", "обрат"
    ],

    # Ветпрепараты / медикаменты (требуют учёта и контроля; не классифицируем как корм)
    "medications": [
        "ампролиум", "amprolium"
    ],

    # Препараты/средства для профилактики метаболических проблем и повышения устойчивости
    "preventive_supplements": [
        "провилит", "провилит (provilit)", "кальвобустер", "кальвобустер (calvobooster)",
        "электролит", "электролитный раствор", "вит-премикс", "мин-премикс", "премикс"
    ]
}

    def classify_ingredient(name):
        if not isinstance(name, str):
            return "other"
        name_lower = name.lower()
        for group, kat_list in GROUP_KEYWORDS.items():
            for kat in kat_list:
                if kat in name_lower:
                    return group
        return "other"
    feeding_and_details['ingredient_group'] = feeding_and_details['decoded_name'].apply(classify_ingredient)
    feeding_and_details = feeding_and_details[feeding_and_details["ingredient_group"] != "other"]

    return feeding_and_details

feeding_and_details = group(feeding_and_details)


Тест на каузальность грейнджера

In [19]:
feeding_and_details.ingredient_group.value_counts()

ingredient_group
forage_bulk                        5943750
energy_source                      2857061
protein_source                     2575414
compound_feed_calf_replacements    1006319
wet_ingredients_water               968343
byproducts                          962608
preventive_supplements              859466
minerals_vitamins                   430888
fat_oils                            232895
buffers_ph                          123232
feed_additives_probiotics           101612
electrolytes                         56841
medications                             38
metabolic_support                        3
Name: count, dtype: int64

In [20]:
feeding_and_details.to_csv("FeedingAndDetails.csv")

In [95]:
others = feeding_details_all[feeding_details_all['ingredient_group'] == 'other']
others['decoded_name'].value_counts()

decoded_name
    4
Name: count, dtype: int64

In [ ]:
others[others["decoded_name"] == ""]

,FeedingTaskID,SectionID,PhysiologicalGroupID,PhysiologicalGroupName,IngredientID,IngredientName,IngredientType,PhysicalWeight_kg,year,norm_ingr_name,is_code,decoded_name,ingredient_group
11574693,Farms/EkoNiva1C.9d4645a7-570c-11e2-9cc0-00155d...,342,54,Т1 (3-5 мес.),260,NaN,Premixture,0.0,2024,,False,,other
11592293,Farms/EkoNiva1C.9d4645a7-570c-11e2-9cc0-00155d...,342,54,Т1 (3-5 мес.),260,NaN,Premixture,0.0,2024,,False,,other
11608364,Farms/EkoNiva1C.9d4645a7-570c-11e2-9cc0-00155d...,342,54,Т1 (3-5 мес.),260,NaN,Premixture,0.0,2024,,False,,other
11622461,Farms/EkoNiva1C.9d4645a7-570c-11e2-9cc0-00155d...,342,54,Т1 (3-5 мес.),260,NaN,Premixture,0.0,2024,,False,,other


In [109]:
feeding_details_all

,FeedingTaskID,SectionID,PhysiologicalGroupID,PhysiologicalGroupName,IngredientID,IngredientName,IngredientType,PhysicalWeight_kg,year,norm_ingr_name,is_code,decoded_name,ingredient_group
0,Farms/EkoNiva1C.216d4235-2852-11e8-80c4-1c98ec...,9,2,Д1,125,Солома покупная,Forage,150.259,2022,солома покупная,False,солома покупная,forage
1,Farms/EkoNiva1C.216d4235-2852-11e8-80c4-1c98ec...,9,2,Д1,773,3645.01.01.01.1.20,Forage,880.444,2022,3645.01.01.01.1.20,True,люцерна сенаж,forage
2,Farms/EkoNiva1C.216d4235-2852-11e8-80c4-1c98ec...,9,2,Д1,72,Комбикорм 10 группы,Concentrate,542.257,2022,комбикорм 10 группы,False,комбикорм 10 группы,premix_blend
3,Farms/EkoNiva1C.216d4235-2852-11e8-80c4-1c98ec...,9,2,Д1,82,Кукуруза сухая,Concentrate,991.793,2022,кукуруза сухая,False,кукуруза сухая,energy
4,Farms/EkoNiva1C.216d4235-2852-11e8-80c4-1c98ec...,9,2,Д1,129,Шрот подсолнечный,Concentrate,156.054,2022,шрот подсолнечный,False,шрот подсолнечный,protein
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16443074,Farms/EkoNiva1C.6c69fe1b-2b6f-11e8-80c4-1c98ec...,131,1,Нетели,119,Солома (общ.),Forage,120.643,2025,солома (общ.),False,солома (общ.),forage
16443075,Farms/EkoNiva1C.6c69fe1b-2b6f-11e8-80c4-1c98ec...,131,1,Нетели,335,6203.01.01.01.1.25,Forage,549.169,2025,6203.01.01.01.1.25,True,люцерна сенаж,forage
16443076,Farms/EkoNiva1C.6c69fe1b-2b6f-11e8-80c4-1c98ec...,131,1,Нетели,331,Жом свекловичный сухой,Concentrate,36.676,2025,жом свекловичный сухой,False,жом свекловичный сухой,byproduct
16443077,Farms/EkoNiva1C.6c69fe1b-2b6f-11e8-80c4-1c98ec...,131,1,Нетели,143,Премикс молодняк 6-24,VitaminMineral,10.049,2025,премикс молодняк 6-24,False,премикс молодняк 6-24,premix


In [112]:
feeding_details_all = feeding_details_all[feeding_details_all["ingredient_group"] != "other"]

In [113]:
feeding_details_all.to_csv("out.csv")